In [98]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.linear_model import LogisticRegression


In [99]:
#data_preprocessing

def data_preprocessing(dataset:pd.DataFrame):
    '''
    dataset contain this columns:
        'gender':               ['Female' 'Male']
        'SeniorCitizen':        [0 1]
        'Partner':              ['Yes' 'No']
        'Dependents':           ['No' 'Yes']
        'tenure':               int
        'PhoneService':         ['No' 'Yes']
        'MultipleLines':        ['No phone service' 'No' 'Yes']
        'InternetService':      ['DSL' 'Fiber optic' 'No']
        'OnlineSecurity':       ['No' 'Yes' 'No internet service']
        'OnlineBackup':         ['Yes' 'No' 'No internet service']
        'DeviceProtection':     ['No' 'Yes' 'No internet service']
        'TechSupport':          ['No' 'Yes' 'No internet service']
        'StreamingTV':          ['No' 'Yes' 'No internet service']
        'StreamingMovies':      ['No' 'Yes' 'No internet service']
        'Contract':             ['Month-to-month' 'One year' 'Two year']
        'PaperlessBilling':     ['Yes' 'No']
        'PaymentMethod':        ['Electronic check' 'Mailed check' 'Bank transfer (automatic)' 'Credit card (automatic)']
        'MonthlyCharges':       float
        'TotalCharges':         float
        'Churn':                ['No' 'Yes']   // Not necessary
    '''
    df = dataset.copy()

    # Prepare string variables
    df.columns = df.columns.str.lower().str.replace(' ','_')
    string_columns = list(df.dtypes[df.dtypes=='O'].index)
    for col in string_columns:
        df[col] = df[col].str.lower().str.replace(' ','_')

    # drop customerid
    if 'customerid' in df.columns:
        del df['customerid']
        
    # Prepare numeric columns type
    if 'tenure' in df.columns:
        df['tenure'] = pd.to_numeric(df['tenure'], errors='coerce')
        df['tenure'] = df['tenure'].fillna(0)
    else:
        raise Exception("!!! Tenure column didn't exist !!!")

    if 'monthlycharges' in df.columns:
        df['monthlycharges'] = pd.to_numeric(df['monthlycharges'], errors='coerce')
        df['monthlycharges'] = df['monthlycharges'].fillna(0)
    else:
        raise Exception("!!! MonthlyCharges column didn't exist !!!")

    if 'totalcharges' in df.columns:
        df['totalcharges'] = pd.to_numeric(df['totalcharges'], errors='coerce')
        df['totalcharges'] = df['totalcharges'].fillna(0)
    else:
        raise Exception("!!! TotalCharges column didn't exist !!!")
        
    # Make dependent variable numeric
    if 'churn' in df.columns:
        df.churn = (df.churn == 'yes').astype(int)
    
    return df

In [100]:
# FeatureEngineering

def FeatureEngineering(dataset:pd.DataFrame, categorical_variables:list, numerical_variables:list, isTrain:bool, scaleColumns:list):
    """
    dataset: pd.Dataframe
    categorical_variables: list of important categorical columns that we want to use in pur model
    numerical_variables: list of important numerical columns that we want to use in pur model
    isTrain:    if you want to train a model, set it 'True'
                if you want to use for test or deployment set it 'False'
    """
    df = dataset.copy()
    categorical_important = categorical_variables.copy()
    numerical_important = numerical_variables.copy()

    df = df[numerical_important + categorical_important]

    # Dummy variables
    if isTrain:
        def ohe(dataset:pd.DataFrame, categorical_columns:str):
            df = dataset.copy()
            df_info = pd.DataFrame(columns=['col', 'uniques'])
            for col in categorical_columns:
                list_Uniques = df[col].unique()
                df_info.loc[len(df_info)] = [col, ','.join(map(str, list_Uniques))]
                for name in list_Uniques:
                    df[str(col)+'_'+str(name)] = df[col].apply(lambda x : 1 if x==name else 0)
                del df[col]

            df_info.to_csv('assets/ohe_info.csv', index=False)
            return df

        df=ohe(df, categorical_important)
    else:
        df_info = pd.read_csv('assets/ohe_info.csv')
        for i in range(len(df_info)):
            col = df_info.iloc[i,:].col
            list_Uniques = df_info.iloc[i,:].uniques.split(',')
            for name in list_Uniques:
                df[str(col)+'_'+str(name)] =  df[col].apply(lambda x : 1 if x==name else 0)
            del df[col]

    # Feature scaling
    if isTrain:
        scale_info = pd.DataFrame(columns=['col','mean','std'])
        for col in scaleColumns:
            mean_col = df[col].mean()
            std_col = df[col].std()
            df[col] = (df[col]-mean_col)/std_col
            scale_info.loc[len(scale_info)] = [col, mean_col, std_col]
        
        scale_info.to_csv('assets/scale_info.csv', index=False)
        # scaler = StandardScaler()
        # scaler.fit(df)
        # df = scaler.transform(df)
        # df = pd.DataFrame(df, columns=scaler.feature_names_in_)
        # pickle.dump(scaler, open('assets/scaler.pkl','wb'))
    else:
        scale_info = pd.read_csv('assets/scale_info.csv')
        
        for i in range(len(scale_info)):
            col = scale_info.loc[i,'col']
            mean_col = scale_info.loc[i,'mean']
            std_col = scale_info.loc[i,'std']
            df[col] = (df[col]-mean_col)/std_col
        # scaler = pickle.load(open('assets/scaler.pkl','rb'))
        # df = scaler.transform(df)
        # df = pd.DataFrame(df, columns=scaler.feature_names_in_)

    return df

In [101]:
df = pd.read_csv('Churn_prediction.csv')
df.drop_duplicates(inplace=True)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df.loc[:,df.columns!='Churn'],
                                                    df['Churn'],
                                                    test_size=0.2,
                                                    random_state=6)

X_test.reset_index(inplace=True,drop=True)
y_test = [1 if x == 'Yes' else 0 for x in y_test]

<h2>Test</h2>

In [102]:
categorical_important = ['contract', 'onlinesecurity', 'techsupport', 'internetservice']
numerical_important = ['tenure', 'monthlycharges', 'totalcharges']

X_test = data_preprocessing(X_test)
X_test = FeatureEngineering(X_test,
                            categorical_variables=categorical_important,
                            numerical_variables=numerical_important,
                            isTrain=False,
                            scaleColumns=numerical_important)


filename = 'assets/clf_Model.sav'
clf_LogisticRegression = pickle.load(open(filename, 'rb'))

pred = clf_LogisticRegression.predict(X_test)



tenure
32.075789847355345
24.492631185276583
monthlycharges
64.6711661341853
30.177218081075942
totalcharges
2255.1279641462547
2260.103006974276


In [103]:
from sklearn.metrics import roc_auc_score, fbeta_score
roc_auc_score(y_test, pred)

0.7027180158751369

In [104]:
X_test

,tenure,monthlycharges,totalcharges,contract_two_year,contract_month-to-month,contract_one_year,onlinesecurity_no_internet_service,onlinesecurity_no,onlinesecurity_yes,techsupport_no_internet_service,techsupport_no,techsupport_yes,internetservice_no,internetservice_fiber_optic,internetservice_dsl
0,1.425907,0.498019,1.344307,1,0,0,0,0,1,0,1,0,0,0,1
1,-1.023810,-0.865592,-0.873911,1,0,0,0,0,1,0,1,0,0,0,1
2,1.180935,1.119349,1.721192,0,0,1,0,1,0,0,0,1,0,1,0
3,0.446020,0.468195,0.533238,1,0,0,0,0,1,0,0,1,0,0,1
4,0.037734,0.792944,0.280329,0,1,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1404,1.058449,-0.303910,0.403421,1,0,0,0,1,0,0,0,1,0,0,1
1405,1.589221,1.788728,2.753181,1,0,0,0,0,1,0,0,1,0,1,0
1406,1.099278,0.481450,1.033237,1,0,0,0,1,0,0,1,0,0,1,0
1407,1.385078,-1.490236,-0.443045,1,0,0,1,0,0,1,0,0,1,0,0
